**一个section接一个section。上个section的数据是这个section的开头**

In [9]:
%%bash
pip3 install user_agents
pip3 install tqdm
pip3 install tables

In [34]:
import numpy as np
import pandas as pd
import pickle
import urllib
from tqdm import tqdm, trange, tqdm_pandas
import os
from user_agents import parse
import datetime
import time
import math
import re
import pytz
tqdm.pandas(desc="Progress Bar")
pd.set_option('display.max_columns', 1000)

---
# Bigquery to Datalab
---
(This part will not be run unless introduce new dataset)

**Load bigquery P3_Ultra_Data_P1**

In [5]:
%%time
# see if define dictionary would solve the problem
Base_Data_table = pd.read_gbq('SELECT * FROM chet_treepie.P3_Ultra_Data_1_10_S1', project_id='cloudiq-treepie-development',dialect='standard')
with open( "../Data/P3_Ultra_Data_1_10_S1.p", "wb" ) as f:
    pickle.dump(Base_Data_table, f)

Requesting query... ok.
Query running...
  Elapsed 10.99 s. Waiting...
  Elapsed 21.51 s. Waiting...
  Elapsed 31.98 s. Waiting...
  Elapsed 42.46 s. Waiting...
  Elapsed 52.95 s. Waiting...
Query done.
Processed: 903.6 Mb

Retrieving results...
  Got page: 1; 1% done. Elapsed 60.0 s.
  Got page: 2; 2% done. Elapsed 65.13 s.
  Got page: 3; 2% done. Elapsed 70.34 s.
  Got page: 4; 3% done. Elapsed 76.3 s.
  Got page: 5; 4% done. Elapsed 82.46 s.
  Got page: 6; 5% done. Elapsed 87.87 s.
  Got page: 7; 6% done. Elapsed 92.79 s.
  Got page: 8; 7% done. Elapsed 97.96 s.
  Got page: 9; 7% done. Elapsed 103.35 s.
  Got page: 10; 8% done. Elapsed 109.76 s.
  Got page: 11; 9% done. Elapsed 116.02 s.
  Got page: 12; 10% done. Elapsed 121.67 s.
  Got page: 13; 11% done. Elapsed 127.09 s.
  Got page: 14; 11% done. Elapsed 132.59 s.
  Got page: 15; 12% done. Elapsed 137.9 s.
  Got page: 16; 13% done. Elapsed 142.49 s.
  Got page: 17; 14% done. Elapsed 148.16 s.
  Got page: 18; 15% done. Elapsed 152

In [6]:
data=Base_Data_table.copy()
data.head(3)

,index,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,42033,4v7xg2f8e6m4fk0kdgh8a3oc,1e8cd1ecab614c1fdd54c12c294a6a0f,2017-06-11 14:50:11,11,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fmyaccount...,NaT,NaT,None,Powerful LED Duracell Torch,92%,1.99,None,ramani@leanage.net,None,1497192146014,None,0.00,False,paypalExpress,True,2017-06-11 15:42:28,2017-06-11 15:56:02,1497194828,80.43.58.192,Mozilla%2F5.0+%28Windows+NT+6.2%3B+rv%3A49.0%2...,None,GB
1,42033,4v7xg2f8e6m4fk0kdgh8a3oc,1e8cd1ecab614c1fdd54c12c294a6a0f,2017-06-11 14:42:28,11,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-06-11 14:42:26.036,None,Powerful LED Duracell Torch,92%,1.99,None,ramani@leanage.net,None,1497192146014,None,0.00,False,paypalExpress,True,2017-06-11 15:42:28,2017-06-11 15:56:02,1497194828,80.43.58.192,Mozilla%2F5.0+%28Windows+NT+6.2%3B+rv%3A49.0%2...,None,GB
2,42033,4v7xg2f8e6m4fk0kdgh8a3oc,1e8cd1ecab614c1fdd54c12c294a6a0f,2017-06-11 14:42:43,11,,,NaT,2017-06-11 14:42:40.244,None,Powerful LED Duracell Torch,92%,1.99,None,ramani@leanage.net,None,1497192146014,None,0.00,False,paypalExpress,True,2017-06-11 15:42:28,2017-06-11 15:56:02,1497194828,80.43.58.192,Mozilla%2F5.0+%28Windows+NT+6.2%3B+rv%3A49.0%2...,None,GB


**Get Size of Base_Data_table**

In [7]:
a=os.path.getsize('../Data/P3_Ultra_Data_1_10_S1.p')/1024/1024/1024
print('Size of P3_Ultra_Data_1_10_S1',a,'GB')

Size of P3_Ultra_Data_1_10_S1 1.1139933867380023 GB


---
# I. Data Initialization
---

## 1. Load data

In [57]:
%%time
with open( "../Data/P3_Ultra_Data_1_10_S1.p", "rb" ) as f:
    data=pickle.load(f)

CPU times: user 4.5 s, sys: 1.81 s, total: 6.31 s
Wall time: 11 s


In [58]:
print('Size of data is: ',os.path.getsize('../Data/P3_Ultra_Data_1_10_S1.p')/1024/1024/1024,'GB')
print('shape of data is: ', data.shape)

Size of data is:  1.1139933867380023 GB
shape of data is:  (1469185, 29)


In [59]:
print(data.shape)
data.head(3)

(1469185, 29)


,index,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,42033,4v7xg2f8e6m4fk0kdgh8a3oc,1e8cd1ecab614c1fdd54c12c294a6a0f,2017-06-11 14:50:11,11,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fmyaccount%2Fwallet,NaT,NaT,None,Powerful LED Duracell Torch,92%,1.99,None,ramani@leanage.net,None,1497192146014,None,0.00,False,paypalExpress,True,2017-06-11 15:42:28,2017-06-11 15:56:02,1497194828,80.43.58.192,Mozilla%2F5.0+%28Windows+NT+6.2%3B+rv%3A49.0%29+Gecko%2F20100101+Firefox%2F49.0+SeaMonkey%2F2.46,None,GB
1,42033,4v7xg2f8e6m4fk0kdgh8a3oc,1e8cd1ecab614c1fdd54c12c294a6a0f,2017-06-11 14:42:28,11,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%2F4031912%3Fito%3Dwowcher_cloudiq_desktop_4031912_bboxdtly8cuq1igi4ohcbmavf%26cloudiq%3D010115%26pay_deposit%3Dfalse%26quantity%3D1%26product_id%3D362590%23cloudiq_session_id%3Dbboxdtly8cuq1igi4ohcbmavf,NaT,2017-06-11 14:42:26.036,None,Powerful LED Duracell Torch,92%,1.99,None,ramani@leanage.net,None,1497192146014,None,0.00,False,paypalExpress,True,2017-06-11 15:42:28,2017-06-11 15:56:02,1497194828,80.43.58.192,Mozilla%2F5.0+%28Windows+NT+6.2%3B+rv%3A49.0%29+Gecko%2F20100101+Firefox%2F49.0+SeaMonkey%2F2.46,None,GB
2,42033,4v7xg2f8e6m4fk0kdgh8a3oc,1e8cd1ecab614c1fdd54c12c294a6a0f,2017-06-11 14:42:43,11,,,NaT,2017-06-11 14:42:40.244,None,Powerful LED Duracell Torch,92%,1.99,None,ramani@leanage.net,None,1497192146014,None,0.00,False,paypalExpress,True,2017-06-11 15:42:28,2017-06-11 15:56:02,1497194828,80.43.58.192,Mozilla%2F5.0+%28Windows+NT+6.2%3B+rv%3A49.0%29+Gecko%2F20100101+Firefox%2F49.0+SeaMonkey%2F2.46,None,GB


**Rename index as id**

In [60]:
data.rename(columns={'index': 'id'}, inplace=True)

## 2. Sort data by id and datetime

In [61]:
data=data.sort_values(['id','datetime'])

**Reset Index**

In [62]:
data.reset_index(drop=True,inplace=True)

In [63]:
list = np.concatenate((range(1,5),range(-3,0)),axis=0)
print(data.shape)
data.iloc[list, :]

(1469185, 29)


,id,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
1,1,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%2F3838997%3Fcloudiq%3D010115%26ito%3Dwowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i%23cloudiq_session_id%3D1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_1+like+Mac+OS+X%29+AppleWebKit%2F603.1.30+%28KHTML%2C+like+Gecko%29+Version%2F10.0+Mobile%2F14E304+Safari%2F602.1,None,GB
2,1,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%2F4096138%2Fblackout-curtains-9-99%3FLinkpos%3DTitle%26blockpos%3D19%26deal%3D4096138%26ito%3Dwowcher_email_National_Personalised_v3%2525202017-06-02%26cauth%3D8a1322f0-af32-446f-8004-2ff26696be4d%26st_cid%3D9751833%26st_rid%3D20405273,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_1+like+Mac+OS+X%29+AppleWebKit%2F603.1.30+%28KHTML%2C+like+Gecko%29+Version%2F10.0+Mobile%2F14E304+Safari%2F602.1,None,GB
3,1,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%2F4096138%2Fblackout-curtains-9-99%3FLinkpos%3DView%26blockpos%3D1%26deal%3D4096138%26ito%3Dwowcher_email_National_Personalised_v3%2525202017-06-09%26cauth%3D8a1322f0-af32-446f-8004-2ff26696be4d%26st_cid%3D9805080%26st_rid%3D20405273,NaT,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,None,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_1+like+Mac+OS+X%29+AppleWebKit%2F603.1.30+%28KHTML%2C+like+Gecko%29+Version%2F10.0+Mobile%2F14E304+Safari%2F602.1,None,GB
4,2,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%2Ftravel%2F4009686%2F69pp-weekender-breaks-3nt-4-algarve%3FLinkpos%3DImage%26blockpos%3D9%26deal%3D4009686%26ito%3Dwowcher_email_Travel_26052017%26cauth%3D1101ee6f-9a13-4c30-9d35-05b605b48795%26st_cid%3D9703436%26st_rid%3D5555079,NaT,2017-05-26 13:25:20.891,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,None,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla%2F5.0+%28Windows+NT+6.0%3B+WOW64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F49.0.2623.112+Safari%2F537.36,None,GB
1469182,52433,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftravel%2F3997899%2F3-4-5-night-new-york-flights-369pp-6-airports,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_1+like+Mac+OS+X%29+AppleWebKit%2F602.4.6+%28KHTML%2C+like+Gecko%29+Version%2F10.0+Mobile%2F14D27+Safari%2F602.1,Oxford,GB
1469183,52433,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftravel%2F3997899%2F3-4-5-night-new-york-flights-369pp-6-airports,NaT,2017-06-12 

## 3. Group by id and session_id

In [67]:
# 这里必须用sort=False，不然会按照session_id的字母排列，打乱时间顺序
df0=data.groupby(['id','session_id'],sort=False).nth(0)

In [16]:
data.columns

Index(['id', 'session_id', 'finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country'],
      dtype='object')

In [17]:
df=df[['finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country']]

In [18]:
df['num_sess']=1
num_row=5
list = np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
print(df.shape)
df.iloc[list, :]

(220047, 28)


finger_print  \
id    session_id                                                     
1     k79c9lk1nfop9m4wx0fmrg16    5c6eaac3c9db786d83c28c4d673ff556   
      w2gunkrgoifz0j4agqb8bz4     5c6eaac3c9db786d83c28c4d673ff556   
      14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   
      1vf4mz1acp57x1p1zlyz5xebe6  5c6eaac3c9db786d83c28c4d673ff556   
2     qvsw8ty43cbb15tu52ujywjs3   e5f3ca057208f51a2cacae58ff95d757   
52432 1waht5m3hhnwa1pvz00evaj2ac  5645078a67cde02aea81f06f54e9728e   
52433 zxkbfysuold2vl2v4el3ubzv    17e96f528f8940c9392619c58b8b136d   
      vxhv5kka1iu93rtmz4g6osc4    de6f52700a2fefcd15e803abbe23cde1   
      15ox967rdf7q3yjn65eedevsh   de6f52700a2fefcd15e803abbe23cde1   
      1tz5t8tsg2bdj5c3w6i1r5ufu   3a131efaa20b6a197508313f0eb266c4   

                                            datetime  days  \
id    session_id                                             
1     k79c9lk1nfop9m4wx0fmrg16   2017-05-19 08:02:53    19   
      w2gunkrgoifz0j4agqb8bz4    2017-05-19 13:51:59    19   
      14qy38xtlrcnc1lamkqq6fqd84 2017-06-02 21:40:06     2   
      1vf4mz1acp57x1p1zlyz5xebe6 2017-06-09 14:23:10     9   
2     qvsw8ty43cbb15tu52ujywjs3  2017-05-26 13:25:27    26   
52432 1waht5m3hhnwa1pvz00evaj2ac 2017-05-20 20:26:03    20   
52433 zxkbfysuold2vl2v4el3ubzv   2017-05-24 18:48:37    24   
      vxhv5kka1iu93rtmz4g6osc4   2017-06-12 14:03:52    12   
      15ox967rdf7q3yjn65eedevsh  2017-06-12 14:03:52    12   
      1tz5t8tsg2bdj5c3w6i1r5ufu  2017-06-12 14:15:41    12   

                                                                           referrer  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16                                                        
      w2gunkrgoifz0j4agqb8bz4                                                         
      14qy38xtlrcnc1lamkqq6fqd84                                                      
      1vf4mz1acp57x1p1zlyz5xebe6                                                      
2     qvsw8ty43cbb15tu52ujywjs3                                                       
52432 1waht5m3hhnwa1pvz00evaj2ac          android-app%3A%2F%2Fcom.google.android.gm   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu                                                       

                                                                           page_url  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      w2gunkrgoifz0j4agqb8bz4     https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      14qy38xtlrcnc1lamkqq6fqd84  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1vf4mz1acp57x1p1zlyz5xebe6  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
2     qvsw8ty43cbb15tu52ujywjs3   https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
52432 1waht5m3hhnwa1pvz00evaj2ac  https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Flondon   

                                 product_viewed_browse  \
id    session_id                                         
1     k79c9lk1nfop9m4wx0fmrg16                     NaT   
      w2gunkrgoifz0j4agqb8bz4                      NaT   
      14qy38xtlrcnc1lamkqq6fqd84                   NaT   
      1vf4mz1acp57x1p1zlyz5xebe6                   NaT   
2     qvsw8ty43cbb15tu52ujywjs

In [19]:
df.loc[1,['finger_print']]

,finger_print
session_id,
k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556
w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556
14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556
1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556


## 4. Add num_sess column
**num_sess is timely sequential**

### 1. Add num_sess to 'df'

In [20]:
# length of the file
tmp=df.iloc[-1,:].name[0]
tmp

52433

In [21]:
# add_num_sess function for df
def add_num_sess(df):
    for i in trange(1, tmp+1):
        df.loc[i,'num_sess']=range(1, df.loc[i,['num_sess']].shape[0]+1)

In [22]:
add_num_sess(df)

100%|██████████| 52433/52433 [04:42<00:00, 185.30it/s]


In [23]:
num_row=5
df=df[['num_sess','finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country']]
list = np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
print(df.shape)
df.iloc[list, :]

(220047, 28)


num_sess                      finger_print  \
id    session_id                                                               
1     k79c9lk1nfop9m4wx0fmrg16           1  5c6eaac3c9db786d83c28c4d673ff556   
      w2gunkrgoifz0j4agqb8bz4            2  5c6eaac3c9db786d83c28c4d673ff556   
      14qy38xtlrcnc1lamkqq6fqd84         3  5c6eaac3c9db786d83c28c4d673ff556   
      1vf4mz1acp57x1p1zlyz5xebe6         4  5c6eaac3c9db786d83c28c4d673ff556   
2     qvsw8ty43cbb15tu52ujywjs3          1  e5f3ca057208f51a2cacae58ff95d757   
52432 1waht5m3hhnwa1pvz00evaj2ac         4  5645078a67cde02aea81f06f54e9728e   
52433 zxkbfysuold2vl2v4el3ubzv           1  17e96f528f8940c9392619c58b8b136d   
      vxhv5kka1iu93rtmz4g6osc4           2  de6f52700a2fefcd15e803abbe23cde1   
      15ox967rdf7q3yjn65eedevsh          3  de6f52700a2fefcd15e803abbe23cde1   
      1tz5t8tsg2bdj5c3w6i1r5ufu          4  3a131efaa20b6a197508313f0eb266c4   

                                            datetime  days  \
id    session_id                                             
1     k79c9lk1nfop9m4wx0fmrg16   2017-05-19 08:02:53    19   
      w2gunkrgoifz0j4agqb8bz4    2017-05-19 13:51:59    19   
      14qy38xtlrcnc1lamkqq6fqd84 2017-06-02 21:40:06     2   
      1vf4mz1acp57x1p1zlyz5xebe6 2017-06-09 14:23:10     9   
2     qvsw8ty43cbb15tu52ujywjs3  2017-05-26 13:25:27    26   
52432 1waht5m3hhnwa1pvz00evaj2ac 2017-05-20 20:26:03    20   
52433 zxkbfysuold2vl2v4el3ubzv   2017-05-24 18:48:37    24   
      vxhv5kka1iu93rtmz4g6osc4   2017-06-12 14:03:52    12   
      15ox967rdf7q3yjn65eedevsh  2017-06-12 14:03:52    12   
      1tz5t8tsg2bdj5c3w6i1r5ufu  2017-06-12 14:15:41    12   

                                                                           referrer  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16                                                        
      w2gunkrgoifz0j4agqb8bz4                                                         
      14qy38xtlrcnc1lamkqq6fqd84                                                      
      1vf4mz1acp57x1p1zlyz5xebe6                                                      
2     qvsw8ty43cbb15tu52ujywjs3                                                       
52432 1waht5m3hhnwa1pvz00evaj2ac          android-app%3A%2F%2Fcom.google.android.gm   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu                                                       

                                                                           page_url  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      w2gunkrgoifz0j4agqb8bz4     https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      14qy38xtlrcnc1lamkqq6fqd84  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1vf4mz1acp57x1p1zlyz5xebe6  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
2     qvsw8ty43cbb15tu52ujywjs3   https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
52432 1waht5m3hhnwa1pvz00evaj2ac  https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Flondon   

                                 product_viewed_browse  \
id    session_id                                         
1     k79c9lk1nfop9m4wx0fmrg16                     NaT   
      w2gunkrgoifz0j4agqb8bz4                      NaT   
      

### 2. Store df

In [24]:
%%time
# write data
with open( "../Data/df.p", "wb" ) as f:
    pickle.dump(df, f)

CPU times: user 2.15 s, sys: 604 ms, total: 2.75 s
Wall time: 4.7 s


### 3. Add num_sess to 'data0'

In [25]:
print('reset index of df:')
df0=df.reset_index()
df0.head(6)

reset index of df:


,id,session_id,num_sess,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,k79c9lk1nfop9m4wx0fmrg16,1,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
1,1,w2gunkrgoifz0j4agqb8bz4,2,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
2,1,14qy38xtlrcnc1lamkqq6fqd84,3,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
3,1,1vf4mz1acp57x1p1zlyz5xebe6,4,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,None,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
4,2,qvsw8ty43cbb15tu52ujywjs3,1,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,NaT,2017-05-26 13:25:20.891,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,None,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla%2F5.0+%28Windows+NT+6.0%3B+WOW64%29+Ap...,None,GB
5,2,xe1kopsb1o573zhid3i51eds,2,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:22,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-06-09 13:20:30.130,NaT,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,None,None,1497014430749,None,229,False,None,False,2017-06-09 14:20:23,2017-06-09 14:20:23,1497016328,148.252.216.66,Mozilla%2F5.0+%28Windows+NT+6.0%3B+WOW64%29+Ap...,None,GB


**Inner join of 'data' and 'df':**

In [26]:
%%time
left=data.copy()
right=df0[['id','session_id','num_sess']].copy()
data0 =pd.merge(left,right,how='inner',on=['id', 'session_id'])

CPU times: user 4.33 s, sys: 823 ms, total: 5.15 s
Wall time: 9.57 s


In [27]:
data0.columns

Index(['id', 'session_id', 'finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country',
       'num_sess'],
      dtype='object')

In [28]:
# Move 'num_sess' to second column
data0=data0[['id','num_sess','session_id', 'finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country']]
data0.sort_values(['id','num_sess'],inplace=True)

In [29]:
data0.reset_index(drop=True, inplace=True)

In [30]:
list=np.concatenate((range(0,3),range(data0.shape[0]-3,data0.shape[0])),axis=0)
data0.loc[list,:]

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
1469182,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB
1469183,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB
1469184,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:26,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Ftravel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB


### 4. Store data0
- **data0 is the data with num_sess sorted by datetime**

In [31]:
%%time
# write data
with open( "../Data/data0.p", "wb" ) as f:
    pickle.dump(data0, f)

CPU times: user 14 s, sys: 7.61 s, total: 21.6 s
Wall time: 40.7 s


# II. Decode User_Agent Feature

## 1. Load Data0

In [32]:
%%time
with open( "../Data/data0.p", "rb" ) as f:
    data0=pickle.load(f)

CPU times: user 6.5 s, sys: 2.51 s, total: 9.01 s
Wall time: 21.9 s


## 2. Decode user_agent

**Transform user_agent dataframe into an array:**

In [33]:
%%time
user_agent=np.asarray(data0.loc[:,'user_agent'])
len(user_agent)

CPU times: user 1e+03 µs, sys: 2 ms, total: 3 ms
Wall time: 39 ms


- **Decode user_agent:**

**user_agent is then well decoded and was made into a list.**

In [34]:
%%time
user_agent=data0.user_agent.copy()
user_agent_refined=user_agent.apply(lambda x: urllib.parse.unquote(x))

CPU times: user 18 s, sys: 183 ms, total: 18.2 s
Wall time: 18.3 s


- **Result:**

In [35]:
print('Before user_agent Refine:')
print(np.asarray(data0.loc[0,'user_agent']))
print('---------------------------------------------------------------------------------------------------------------------------------------------------------')
print('After user_agent Refine:')
print(user_agent_refined[1])

Before user_agent Refine:
Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_1+like+Mac+OS+X%29+AppleWebKit%2F603.1.30+%28KHTML%2C+like+Gecko%29+Version%2F10.0+Mobile%2F14E304+Safari%2F602.1
---------------------------------------------------------------------------------------------------------------------------------------------------------
After user_agent Refine:
Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1


## 3. Replace user_agent column
**Transform previous user_agent data from list to dataframe**

In [36]:
user_agent_column=user_agent_refined.copy()
user_agent_column.head()

0    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
1    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
2    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
3    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
4    Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...
Name: user_agent, dtype: object

**assign new user_agent_column value to user_agent value in original dataframe**

In [37]:
data0['user_agent']=user_agent_column.copy()
data0['user_agent'].head()

0    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
1    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
2    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
3    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
4    Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...
Name: user_agent, dtype: object

In [38]:
print(data0.shape)

num_row=5
list = np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
data0.iloc[list, :]

(1469185, 30)


,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
3,1,4,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,None,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
4,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,NaT,2017-05-26 13:25:20.891,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,None,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB
1469180,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:26:01,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Ftravel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469181,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:04,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,2017-06-12 14:28:04.531,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469182,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469183,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.

## 4. Save data_UserAgent:
**data_final is the data such that:**
1. Full entries of all history (before groupby)
2. Sorted by datatime
3. With user_agent column decoded

In [39]:
%%time
with open( "../Data/data_UserAgent.p", "wb" )  as f:
    pickle.dump(data0, f)

CPU times: user 11.2 s, sys: 10.1 s, total: 21.3 s
Wall time: 39.9 s


## 5. Check data:

In [40]:
%%time
with open( "../Data/data_UserAgent.p", "rb" ) as f:
    data_UserAgent=pickle.load(f)

CPU times: user 6.39 s, sys: 2.2 s, total: 8.59 s
Wall time: 25.3 s


In [41]:
data_UserAgent.columns

Index(['id', 'num_sess', 'session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country'],
      dtype='object')

In [42]:
data_UserAgent[['id', 'num_sess', 'session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'product_viewed_cart','cart_started',
       'cart_startTime','cart_firstTime', 'cart_lastTime','basket_paid_browsing', 'basket_paid_cart',
       'basketInfoTime','product_name', 'product_save', 'product_price','product_time_left',  'total',  
       'payment_type','email_click_through', 'email', 'ip_address', 'user_agent','Post_Code','city', 'country']][:3]

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,product_viewed_cart,cart_started,cart_startTime,cart_firstTime,cart_lastTime,basket_paid_browsing,basket_paid_cart,basketInfoTime,product_name,product_save,product_price,product_time_left,total,payment_type,email_click_through,email,ip_address,user_agent,Post_Code,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,None,2017-05-19 08:02:51.292,1495180971292,2017-05-19 09:02:53,2017-05-19 09:02:53,None,False,1495182843,None,None,None,None,None,None,True,None,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,None,2017-05-19 08:02:51.292,1495180971292,2017-05-19 14:51:59,2017-05-19 14:51:59,None,False,1495203782,None,None,None,None,None,None,True,None,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,None,2017-06-02 21:40:06.147,1496439606147,2017-06-02 22:40:06,2017-06-02 22:40:06,None,False,1496441524,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,9,None,False,None,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,None,GB


In [43]:
data_UserAgent[['basket_paid_browsing','basket_paid_cart']][:5]
# data_final['basket_paid_browsing']

,basket_paid_browsing,basket_paid_cart
0,None,False
1,None,False
2,None,False
3,None,False
4,None,False


In [44]:
['basket_paid_browsing','basket_paid_cart']

['basket_paid_browsing', 'basket_paid_cart']

In [45]:
sum(data_UserAgent['basket_paid_browsing'].apply(lambda x: x!=None))

20458

In [46]:
print(sum(data_UserAgent['basket_paid_cart'].apply(lambda x: x!='False')))
print(sum(data_UserAgent['basket_paid_cart'].apply(lambda x: x=='True')))

376486
376486


---
# III. Decode url to product id

## 1. Decode URLs

### 1.1 Load data_UserAgent

In [4]:
%%time
with open( "../Data/data_UserAgent.p", "rb" ) as f:
    data_UserAgent=pickle.load(f)

with open( "../Data/P3_Ultra_Data_1_10_S1.p", "rb" ) as f:
    data=pickle.load(f)

CPU times: user 11 s, sys: 3.89 s, total: 14.9 s
Wall time: 21.6 s


### 1.2 Decode URL

In [5]:
tqdm.pandas(desc="Progress Bar")
Page_URL=data_UserAgent['page_url'].progress_apply(lambda x: urllib.parse.unquote(x))
Referrer=data_UserAgent['referrer'].progress_apply(lambda x: urllib.parse.unquote(x))

Progress Bar: 100%|██████████| 1469185/1469185 [00:13<00:00, 112615.30it/s]


In [6]:
data_DecodeURL=data_UserAgent.copy()
data_DecodeURL['page_url']=Page_URL
data_DecodeURL['referrer']=Referrer

In [ ]:
data_DecodeURL[['referrer','page_url']][:20]

,referrer,page_url
0,,https://secure.wowcher.co.uk/checkout/3838997?...
1,,https://secure.wowcher.co.uk/checkout/3838997?...
2,,https://www.wowcher.co.uk/deal/shop/4096138/bl...
3,,https://www.wowcher.co.uk/deal/shop/4096138/bl...
4,,https://www.wowcher.co.uk/email-deals/travel/4...
5,,https://www.wowcher.co.uk/email-deals/travel/4...
6,,https://www.wowcher.co.uk/email-deals/travel/4...
7,,https://www.wowcher.co.uk/email-deals/travel/4...
8,,https://www.wowcher.co.uk/deal/shop/4212459/mo...
9,,https://www.wowcher.co.uk/deal/shop/4212459/mo...


### 1.3 Save data_DecodeURL
1. Full entries of all history (before groupby)
2. Sorted by datatime
3. With user_agent column decoded
4. With All URL decoded

In [ ]:
%%time
with open( "../Data/data_DecodeURL.p", "wb" )  as f:
    pickle.dump(data_DecodeURL, f)

## 2. Product_id with product_viewed!=None 
**product_viewed_browse**

### 2.1 Load data_DecodeURL

In [39]:
%%time
with open( "../Data/data_DecodeURL.p", "rb" ) as f:
    data_DecodeURL=pickle.load(f)

with open( "../Data/P3_Ultra_Data_1_10_S1.p", "rb" ) as f:
    data=pickle.load(f)

CPU times: user 9.39 s, sys: 4.21 s, total: 13.6 s
Wall time: 18.1 s


In [44]:
data_DecodeURL.shape

(1469185, 30)

In [45]:
data_DecodeURL[:3]

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=Title&blockpos=19&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-02&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9751833&st_rid=20405273,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB


In [43]:
pd.set_option('display.max_colwidth', -1)

tmp0 = data_DecodeURL['product_viewed_browse'].isnull()
# data_final such that there's a product view for that browse:
# 就是取非null的product_viewed_browse的对应URL，作为url_product_viewed.
url_product_viewed=data_DecodeURL[tmp0==False]['page_url']
url_product_viewed[:3]

5    https://www.wowcher.co.uk/email-deals/travel/4009686/69pp-weekender-breaks-3nt-4-algarve?Linkpos=Image&blockpos=9&deal=4009686&ito=wowcher_email_Travel_26052017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9703436&st_rid=5555079          
6    https://www.wowcher.co.uk/email-deals/travel/4085525/7nt-4-star-inclusive-dubrovnik-croatia-break?Linkpos=Image&blockpos=10&deal=4085525&ito=wowcher_email_Travel_09062017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9805752&st_rid=5555079
8    https://www.wowcher.co.uk/deal/shop/4212459/modern-shaggy-rug-9?Linkpos=Title&blockpos=1&deal=4212459&ito=wowcher_email_National_Default%25202017-06-12&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9822925&st_rid=5555079                   
Name: page_url, dtype: object

### 2.3 Extract product_id from URL

**Function to find product_id:**

In [47]:
def find_product_id(x):
    try:
        return re.search(r'\d{7}', x).group()
    except AttributeError:
        return x
# re.search(r'\d+', url[5]).group()
# re is a package

**A small test of RegExp:**

In [48]:
print(re.search(r'\d{7}', '123456789/@90123456789').group())

1234567


**Official extract:**

In [49]:
product_id = url_product_viewed.apply(lambda x:find_product_id(x))
print('length of both valid and Non-valid product id:', product_id.shape)
product_id[:16]
# 非null的情况下会有找不出productid的情况

length of both valid and Non-valid product id: (419541,)


5     4009686                                                             
6     4085525                                                             
8     4212459                                                             
10    4247216                                                             
12    4009062                                                             
14    4085938                                                             
17    4085938                                                             
20    4095939                                                             
21    4095939                                                             
23    4095939                                                             
24    4031765                                                             
32    4042013                                                             
35    4042013                                                             
37    https://www.wowcher

**Function: count number of valid product_id:**

In [52]:
#下面这个就是把不能提取初product_id的删掉，不数
def count_product_id(x):
    try:
        int(x)
        return 1
    except ValueError:
        return 0

In [53]:
print('number of valid product id:',sum(product_id.apply(lambda x: count_product_id(x))))
print('number of all product id  :', product_id.shape[0])

number of valid product id: 265469
number of all product id  : 419541


**One example of not having product_id**
<br> Customer searched the product name. And on the result page, he could directly go to buy stage. Or yes, he is viewing a product page, where we didn't record, as all info has been on the result page already.

In [54]:
url_product_viewed[37]

'https://www.wowcher.co.uk/deals/travel/spas-country-house/spa-breaks'

**Substitute unvalid url to '0'**

In [56]:
# 把不能提取product_id的非null的都替换成0
def to_0(x):
    try:
        int(x)
        return x
    except ValueError:
        return 0

In [57]:
product_id_final=product_id.apply(lambda x:to_0(x))
print(product_id_final.shape)
product_id_final[:16]

(419541,)


5     4009686
6     4085525
8     4212459
10    4247216
12    4009062
14    4085938
17    4085938
20    4095939
21    4095939
23    4095939
24    4031765
32    4042013
35    4042013
37    0      
39    3701601
43    4031709
Name: page_url, dtype: object

### 2.4 Add product_id_ProductViewed

In [58]:
data_ProductID=data_DecodeURL.copy()
data_ProductID['product_id_ProductViewed']=product_id_final

In [59]:
data_ProductID.columns

Index(['id', 'num_sess', 'session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country',
       'product_id_ProductViewed'],
      dtype='object')

**Show product_id_ProductViewed**

In [60]:
num_row=5
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)

data_ProductID[['product_viewed_browse','product_id_ProductViewed']].iloc[list,:]

,product_viewed_browse,product_id_ProductViewed
0,NaT,NaN
1,NaT,NaN
2,NaT,NaN
3,NaT,NaN
4,NaT,NaN
1469180,NaT,NaN
1469181,2017-06-12 14:28:04.531,3997899
1469182,NaT,NaN
1469183,NaT,NaN
1469184,NaT,NaN


### 2.5 Save data_ProductID
**data_ProductID的最后一行是product_id_ProductViewed，原本null的还是null，非null的能**

In [61]:
print('shape of data_final is: ',data_ProductID.shape)
num_row=3
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)

data_ProductID.iloc[list,:]

shape of data_final is:  (1469185, 31)


,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country,product_id_ProductViewed
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=Title&blockpos=19&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-02&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9751833&st_rid=20405273,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN
1469182,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https://www.wowcher.co.uk/deal/travel/3997899/3-4-5-night-new-york-flights-369pp-6-airports,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like+Mac+OS+X)+AppleWebKit/602.4.6+(KHTML,+like+Gecko)+Version/10.0+Mobile/14D27+Safari/602.1",Oxford,GB,NaN
1469183,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https://www.wowcher.co.uk/deal/travel/3997899/3-4-5-night-new-york-flights-369pp-6-airports,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like+Mac+OS+X)+AppleWebKit/602.4.6+(KHTML,+like+Gecko)+Version/10.0+Mobile/14D27+Safari/602.1",Oxford,GB,NaN
1469184,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:26,12,,https://www.wowcher.co.uk/deals/travel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like+Mac+OS+X)+AppleWebKit/602.4.6+(KHTML,+like+Gecko)+Version/10.0+Mobile/14D27+Safari/602.1",Oxford,GB,NaN


In [64]:
%%time
with open( "../Data/data_ProductID.p", "wb" )  as f:
    pickle.dump(data_ProductID, f)

CPU times: user 9.97 s, sys: 2.04 s, total: 12 s
Wall time: 17 s


## 3. Product_id with cart_started!=None 
cart_started is from browse data and always use this!!!

### 3.1 Load data_ProductID

In [17]:
%%time
with open( "../Data/data_ProductID.p", "rb" ) as f:
    data_ProductID=pickle.load(f)

CPU times: user 4.26 s, sys: 1.78 s, total: 6.04 s
Wall time: 6.04 s


In [18]:
data_ProductID['cart_startTime'][:3]

0    1495180971292
1    1495180971292
2    1496439606147
Name: cart_startTime, dtype: object

**Why Choose 'cart_started'**

In [19]:
def delete_3_digit(x):
    return np.int(math.trunc(x/1000))

def to_time(x):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))

In [20]:
# can only be run for 1 time within this section
data_ProductID['cart_startTime']=data_ProductID['cart_startTime'].apply(lambda x: 
                                                to_time(delete_3_digit(int(x))) if not pd.isnull(x) else 0).copy()

We can see from the below table: cart_startTime and cart_firstTime are kind of same thing, they are the time the 1st time a session entered cart.

In [21]:
index=(data_ProductID['id']==52433)
data_ProductID[['id','num_sess','datetime','product_viewed_browse','cart_started','cart_startTime',
                'cart_firstTime','page_url','referrer']][index]

,id,num_sess,datetime,product_viewed_browse,cart_started,cart_startTime,cart_firstTime,page_url,referrer
1469154,52433,1,2017-05-24 18:48:37,NaT,2017-05-24 18:48:36.464,2017-05-24 18:43:00,2017-05-24 19:43:00,https://secure.wowcher.co.uk/checkout/3949698?product_id=356882&quantity=1&pay_deposit=false,https://www.wowcher.co.uk/deal/shop/3997409/9-seater-rattan-corner-sofa-table-set-569
1469155,52433,2,2017-06-12 14:03:52,NaT,2017-06-12 14:03:52.170,2017-06-12 14:03:52,2017-06-12 15:03:52,https://secure.wowcher.co.uk/checkout/4211595?product_id=373532&quantity=1&pay_deposit=false,https://www.wowcher.co.uk/deal/shop/4211595/9-seater-rattan-dining-sofa-set-509?searchresult=true
1469156,52433,3,2017-06-12 14:03:52,NaT,2017-06-12 14:03:52.155,2017-06-12 14:03:52,2017-06-12 15:03:14,https://secure.wowcher.co.uk/checkout/4211595?product_id=373532&quantity=1&pay_deposit=false,https://www.wowcher.co.uk/deal/shop/4211595/9-seater-rattan-dining-sofa-set-509?searchresult=true
1469157,52433,4,2017-06-12 14:15:41,NaT,NaT,2017-06-12 14:16:23,2017-06-12 15:15:54,https://www.wowcher.co.uk/deals/london,
1469158,52433,4,2017-06-12 14:16:01,NaT,NaT,2017-06-12 14:16:23,2017-06-12 15:15:54,https://www.wowcher.co.uk/deals/travel,
1469159,52433,4,2017-06-12 14:16:13,2017-06-12 14:16:13.316,NaT,2017-06-12 14:16:23,2017-06-12 15:15:54,https://www.wowcher.co.uk/deal/travel/3913257/69pp-bargain-late-holidays-night-4-malta-spa,
1469160,52433,4,2017-06-12 14:16:23,NaT,2017-06-12 14:16:23.453,2017-06-12 14:16:23,2017-06-12 15:15:54,https://www.wowcher.co.uk/deal/travel/3913257/69pp-bargain-late-holidays-night-4-malta-spa,
1469161,52433,4,2017-06-12 14:16:58,NaT,NaT,2017-06-12 14:16:23,2017-06-12 15:15:54,https://www.wowcher.co.uk/deals/travel,
1469162,52433,4,2017-06-12 14:17:21,2017-06-12 14:17:21.577,NaT,2017-06-12 14:16:23,2017-06-12 15:15:54,https://www.wowcher.co.uk/deal/travel/4186830/299pp-super-escapes-travel-4-5-seven-night,
1469163,52433,4,2017-06-12 14:17:38,NaT,2017-06-12 14:17:38.942,2017-06-12 14:16:23,2017-06-12 15:15:54,https://secure.wowcher.co.uk/checkout/4186830?product_id=371778&quantity=1&pay_deposit=false,https://www.wowcher.co.uk/deal/travel/4186830/299pp-super-escapes-travel-4-5-seven-night


In [7]:
pd.set_option('display.max_colwidth', -1)
# 取cart_started非0非空的行index
index_cartStarted_nonNull=pd.isnull(data_ProductID['cart_started'])==False
tmp=data_ProductID[['cart_started','page_url']][index_cartStarted_nonNull]
# 就是非空的cart_started对应的page_url
url=tmp['page_url']

In [8]:
url[:6]

0    https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i                                                                          
1    https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i                                                                          
2    https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=Title&blockpos=19&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-02&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9751833&st_rid=20405273      
3    https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=View&blockpos=1&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-09&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9805080&st_rid=20405273        
4   

### 3.2 Extract product_id from URL

In [22]:
def find_product_id(x):
    try:
        return re.search(r'\d{7}', x).group()
    except AttributeError:
        return x

In [23]:
product_id=url.apply(lambda x:find_product_id(x))
print('length of both valid and Non-valid product id:', product_id.shape[0])
product_id[:25]

length of both valid and Non-valid product id: 795708


0     3838997                                                             
1     3838997                                                             
2     4096138                                                             
3     4096138                                                             
4     4009686                                                             
7     4085525                                                             
9     4212459                                                             
11    4247216                                                             
13    4009062                                                             
15    4085938                                                             
16    4085938                                                             
18    4085938                                                             
19    4085938                                                             
22    4095939            

### 3.3 Substitute invalid product_id to '0'

In [24]:
def to_0(x):
    try:
        int(x)
        return x
    except ValueError:
        return 0

In [26]:
product_id_final=product_id.apply(lambda x:to_0(x))
print(product_id_final.shape)
product_id_final[:25]

(795708,)


0     3838997
1     3838997
2     4096138
3     4096138
4     4009686
7     4085525
9     4212459
11    4247216
13    4009062
15    4085938
16    4085938
18    4085938
19    4085938
22    4095939
25    4031765
26    4020941
27    4156807
28    4199613
29    3938113
30    3689409
31    3676110
33    4042013
34    4042013
36    0      
38    0      
Name: page_url, dtype: object

In [27]:
data_ProductID['product_id_CartStarted']=product_id_final.copy()

### 3.4 Save data_ProductID
1. Full entries of all history (before groupby)
2. Sorted by datatime
3. With user_agent column decoded
4. With All URL decoded
5. product_id_ProductViewed & product_id_CartStarted 2 columns added

In [28]:
print('shape of data_final is: ',data_ProductID.shape)
num_row=3
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)

data_ProductID.iloc[list,:]

shape of data_final is:  (1469185, 32)


,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country,product_id_ProductViewed,product_id_CartStarted
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,2017-05-19 08:02:51,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN,3838997
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,2017-05-19 08:02:51,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN,3838997
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=Title&blockpos=19&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-02&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9751833&st_rid=20405273,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,2017-06-02 21:40:06,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN,4096138
1469182,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https://www.wowcher.co.uk/deal/travel/3997899/3-4-5-night-new-york-flights-369pp-6-airports,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,2017-06-12 14:16:23,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like+Mac+OS+X)+AppleWebKit/602.4.6+(KHTML,+like+Gecko)+Version/10.0+Mobile/14D27+Safari/602.1",Oxford,GB,NaN,3997899
1469183,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https://www.wowcher.co.uk/deal/travel/3997899/3-4-5-night-new-york-flights-369pp-6-airports,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,2017-06-12 14:16:23,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like+Mac+OS+X)+AppleWebKit/602.4.6+(KHTML,+like+Gecko)+Version/10.0+Mobile/14D27+Safari/602.1",Oxford,GB,NaN,3997899
1469184,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:26,12,,https://www.wowcher.co.uk/deals/travel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,2017-06-12 14:16:23,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like+Mac+OS+X)+AppleWebKit/602.4.6+(KHTML,+like+Gecko)+Version/10.0+Mobile/14D27+Safari/602.1",Oxford,GB,NaN

In [29]:
data_ProductID[['cart_started','product_id_CartStarted']][:8]

,cart_started,product_id_CartStarted
0,2017-05-19 08:02:51.292,3838997
1,2017-05-19 08:02:51.292,3838997
2,2017-06-02 21:40:06.147,4096138
3,2017-06-09 14:23:10.017,4096138
4,2017-05-26 13:25:20.891,4009686
5,NaT,NaN
6,NaT,NaN
7,2017-06-09 13:20:30.749,4085525


In [30]:
%%time
with open( "../Data/data_ProductID.p", "wb" )  as f:
    pickle.dump(data_ProductID, f)

CPU times: user 10.1 s, sys: 2.6 s, total: 12.7 s
Wall time: 19.8 s


# IV. Convert TimeZone
**Here only cart_firstTime and cart_lastTime are not UTC**

## 1. Load data_ProductID

In [31]:
%%time
with open( "../Data/data_ProductID.p", "rb" ) as f:
    data_ProductID=pickle.load(f)

CPU times: user 4.34 s, sys: 1.69 s, total: 6.03 s
Wall time: 6.03 s


## 2. Convert Timezone

In [35]:
tmp_last=data_ProductID['cart_lastTime'].copy()
tmp_first=data_ProductID['cart_firstTime'].copy()

This is going to take long:

In [36]:
# don't need to use this specify twice in a notebook
# tqdm.pandas(desc="Progress Bar")
New_cart_lastTime=tmp_last.progress_apply(lambda x: (pd.to_datetime(x).tz_localize('Europe/London')).tz_convert('UTC'))
New_cart_firstTime=tmp_first.progress_apply(lambda x: (pd.to_datetime(x).tz_localize('Europe/London')).tz_convert('UTC'))

Progress Bar: 100%|██████████| 1469185/1469185 [04:00<00:00, 6099.71it/s]


In [37]:
data_TimeZone=data_ProductID.copy()
data_TimeZone['cart_lastTime']=New_cart_lastTime
data_TimeZone['cart_firstTime']=New_cart_firstTime

## 3. Save data_TimeZone

In [38]:
%%time
with open( "../Data/data_TimeZone.p", "wb" )  as f:
    pickle.dump(data_TimeZone, f)

CPU times: user 9.49 s, sys: 2.04 s, total: 11.5 s
Wall time: 14.5 s


In [39]:
print('shape of data_TimeZone is: ',data_TimeZone.shape)
num_row=5
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)

data_TimeZone[['id','num_sess','datetime','cart_startTime','cart_started','cart_firstTime','cart_lastTime',
            'basket_paid_cart']].iloc[list,:]

shape of data_TimeZone is:  (1469185, 32)


,id,num_sess,datetime,cart_startTime,cart_started,cart_firstTime,cart_lastTime,basket_paid_cart
0,1,1,2017-05-19 08:02:53,2017-05-19 08:02:51,2017-05-19 08:02:51.292,2017-05-19 08:02:53+00:00,2017-05-19 08:02:53+00:00,False
1,1,2,2017-05-19 13:51:59,2017-05-19 08:02:51,2017-05-19 08:02:51.292,2017-05-19 13:51:59+00:00,2017-05-19 13:51:59+00:00,False
2,1,3,2017-06-02 21:40:06,2017-06-02 21:40:06,2017-06-02 21:40:06.147,2017-06-02 21:40:06+00:00,2017-06-02 21:40:06+00:00,False
3,1,4,2017-06-09 14:23:10,2017-06-09 14:23:10,2017-06-09 14:23:10.017,2017-06-09 14:23:10+00:00,2017-06-09 14:23:10+00:00,False
4,2,1,2017-05-26 13:25:27,2017-05-26 13:25:20,2017-05-26 13:25:20.891,2017-05-26 13:25:27+00:00,2017-05-26 13:25:27+00:00,False
1469180,52433,4,2017-06-12 14:26:01,2017-06-12 14:16:23,NaT,2017-06-12 14:15:54+00:00,2017-06-12 14:28:26+00:00,False
1469181,52433,4,2017-06-12 14:28:04,2017-06-12 14:16:23,NaT,2017-06-12 14:15:54+00:00,2017-06-12 14:28:26+00:00,False
1469182,52433,4,2017-06-12 14:28:09,2017-06-12 14:16:23,2017-06-12 14:28:09.550,2017-06-12 14:15:54+00:00,2017-06-12 14:28:26+00:00,False
1469183,52433,4,2017-06-12 14:28:09,2017-06-12 14:16:23,2017-06-12 14:28:09.554,2017-06-12 14:15:54+00:00,2017-06-12 14:28:26+00:00,False
1469184,52433,4,2017-06-12 14:28:26,2017-06-12 14:16:23,NaT,2017-06-12 14:15:54+00:00,2017-06-12 14:28:26+00:00,False


# V. Redefine cart_started time
Here for each session, we take the final nonNull cell of cart_started and use the related datetime as cart_started

## 1. Load data_TimeZone

In [40]:
%%time
with open( "../Data/data_TimeZone.p", "rb" ) as f:
    data_TimeZone=pickle.load(f)

CPU times: user 4.1 s, sys: 1.41 s, total: 5.51 s
Wall time: 5.51 s


## 2. Preprosess:
<br>先看一下 cart_started time 和同行的datatime的确差的不错

In [41]:
index=(data_ProductID['id']==9)
data_TimeZone[['id','num_sess','datetime','cart_started','cart_firstTime','cart_lastTime',
            'basket_paid_cart']][index]

,id,num_sess,datetime,cart_started,cart_firstTime,cart_lastTime,basket_paid_cart
89,9,1,2017-06-02 18:47:14,2017-06-02 18:47:14.440,2017-06-02 18:47:14+00:00,2017-06-02 18:47:14+00:00,False
90,9,2,2017-06-02 22:01:05,2017-06-02 22:01:05.686,2017-06-02 22:01:05+00:00,2017-06-02 22:01:05+00:00,False
91,9,3,2017-06-04 08:51:02,2017-06-04 08:51:02.535,2017-06-04 08:51:02+00:00,2017-06-04 08:51:02+00:00,False


看看而已。。。

In [45]:
tmp=data_TimeZone.groupby(['id','num_sess']).last()[['cart_started']]

In [46]:
tmp.iloc[-5:,:]

cart_started
id    num_sess                              
52432 4        2017-05-20 20:26:01.808000000
52433 1        2017-05-24 18:48:36.464000000
      2        2017-06-12 14:03:52.169999872
      3        2017-06-12 14:03:52.155000064
      4        2017-06-12 14:28:09.553999872

In [47]:
print('number of all cart_started for data_TimeZone is     : ', data_TimeZone['cart_started'].shape[0])
print('number of cart_started is null for data_TimeZone is : ',sum(data_TimeZone['cart_started'].isnull()))

number of all cart_started for data_TimeZone is     :  1469185
number of cart_started is null for data_TimeZone is :  673477


## 3. Find last Cart_started for all session

In [48]:
#非null的cart_started
nonNull_cart_started=data_TimeZone[data_TimeZone['cart_started'].isnull()==False]
nonNull_cart_started_withIndex=nonNull_cart_started.reset_index()

In [49]:
df_nonNull_cart_started=nonNull_cart_started_withIndex.groupby(['id','num_sess']).nth(-1)
df_nonNull_cart_started[['index','datetime','cart_started']][:10]
#怎么cart_started还是有问题啊。。。感觉有的还是不在一个时区啊？？？不过不要紧，lasttime都已经是UTC。现在这个处理目测是最优解

index            datetime            cart_started
id num_sess                                                   
1  1         0     2017-05-19 08:02:53 2017-05-19 08:02:51.292
   2         1     2017-05-19 13:51:59 2017-05-19 08:02:51.292
   3         2     2017-06-02 21:40:06 2017-06-02 21:40:06.147
   4         3     2017-06-09 14:23:10 2017-06-09 14:23:10.017
2  1         4     2017-05-26 13:25:27 2017-05-26 13:25:20.891
   2         7     2017-06-09 13:20:23 2017-06-09 13:20:30.749
   3         9     2017-06-12 08:32:26 2017-06-12 08:32:44.377
   4         13    2017-06-14 13:51:09 2017-06-14 13:50:47.705
3  1         19    2017-06-03 18:05:39 2017-06-03 17:05:07.754
   2         22    2017-06-05 23:08:27 2017-06-05 22:07:57.983

## 4. Substitute cart_started by datetime

In [50]:
df_nonNull_cart_started['cart_started']=df_nonNull_cart_started['datetime'].copy()

In [51]:
df_tmp=df_nonNull_cart_started[['index','cart_started']].copy()
ttt = df_tmp.reset_index()

In [52]:
data_final=data_TimeZone.copy()
data_final['last_cart_started']=ttt.set_index('index')['cart_started']

In [53]:
print('shape of data_final is   : ',data_final.shape)
print('shape of data_TimeZone is: ',data_TimeZone.shape)
num_row=3
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)

data_final[['datetime','cart_started']].iloc[list,:]

shape of data_final is   :  (1469185, 33)
shape of data_TimeZone is:  (1469185, 32)


,datetime,cart_started
0,2017-05-19 08:02:53,2017-05-19 08:02:51.292
1,2017-05-19 13:51:59,2017-05-19 08:02:51.292
2,2017-06-02 21:40:06,2017-06-02 21:40:06.147
1469182,2017-06-12 14:28:09,2017-06-12 14:28:09.550
1469183,2017-06-12 14:28:09,2017-06-12 14:28:09.554
1469184,2017-06-12 14:28:26,NaT


In [54]:
data_final[['id','num_sess','datetime','cart_started','last_cart_started','page_url','product_id_CartStarted']][:20]

,id,num_sess,datetime,cart_started,last_cart_started,page_url,product_id_CartStarted
0,1,1,2017-05-19 08:02:53,2017-05-19 08:02:51.292,2017-05-19 08:02:53,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,3838997
1,1,2,2017-05-19 13:51:59,2017-05-19 08:02:51.292,2017-05-19 13:51:59,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,3838997
2,1,3,2017-06-02 21:40:06,2017-06-02 21:40:06.147,2017-06-02 21:40:06,https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=Title&blockpos=19&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-02&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9751833&st_rid=20405273,4096138
3,1,4,2017-06-09 14:23:10,2017-06-09 14:23:10.017,2017-06-09 14:23:10,https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=View&blockpos=1&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-09&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9805080&st_rid=20405273,4096138
4,2,1,2017-05-26 13:25:27,2017-05-26 13:25:20.891,2017-05-26 13:25:27,https://www.wowcher.co.uk/email-deals/travel/4009686/69pp-weekender-breaks-3nt-4-algarve?Linkpos=Image&blockpos=9&deal=4009686&ito=wowcher_email_Travel_26052017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9703436&st_rid=5555079,4009686
5,2,1,2017-05-26 13:25:27,NaT,NaT,https://www.wowcher.co.uk/email-deals/travel/4009686/69pp-weekender-breaks-3nt-4-algarve?Linkpos=Image&blockpos=9&deal=4009686&ito=wowcher_email_Travel_26052017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9703436&st_rid=5555079,NaN
6,2,2,2017-06-09 13:20:22,NaT,NaT,https://www.wowcher.co.uk/email-deals/travel/4085525/7nt-4-star-inclusive-dubrovnik-croatia-break?Linkpos=Image&blockpos=10&deal=4085525&ito=wowcher_email_Travel_09062017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9805752&st_rid=5555079,NaN
7,2,2,2017-06-09 13:20:23,2017-06-09 13:20:30.749,2017-06-09 13:20:23,https://www.wowcher.co.uk/email-deals/travel/4085525/7nt-4-star-inclusive-dubrovnik-croatia-break?Linkpos=Image&blockpos=10&deal=4085525&ito=wowcher_email_Travel_09062017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9805752&st_rid=5555079,4085525
8,2,3,2017-06-12 08:32:26,NaT,NaT,https://www.wowcher.co.uk/deal/shop/4212459/modern-shaggy-rug-9?Linkpos=Title&blockpos=1&deal=4212459&ito=wowcher_email_National_Default%25202017-06-12&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9822925&st_rid=5555079,NaN
9,2,3,2017-06-12 08:32:26,2017-06-12 08:32:44.377,2017-06-12 08:32:26,https://www.wowcher.co.uk/deal/shop/4212459/modern-shaggy-rug-9?Linkpos=Title&blockpos=1&deal=4212459&ito=wowcher_email_National_Default%25202017-06-12&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9822925&st_rid=5555079,4212459


## 5. Save df_final
- 按照id，num_sess, datetime排列的
- userAgent解析过了
- decode了所有URL和referrer
- firstTime和lastTime时差转换过了
- 找出product_viewed非空的时候的url，解析product_id_ProductViewed，不能解析的设为0.所以现在product_id_ProductViewed里面有0有nan
- 找出cart_started非空的时候的url，解析product_id_CartStarted，不能解析的设为0.所以现在product_id_CartStarted里面有0有nan
- 至此为止，保留了所有的原始数据，两种product_id也都是针对所有数据来看的，每行都解析

In [55]:
%%time
with open( "../Data/data_final.p", "wb" )  as f:
    pickle.dump(data_final, f)

CPU times: user 9.69 s, sys: 1.95 s, total: 11.6 s
Wall time: 17.7 s


In [56]:
data_final

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country,product_id_ProductViewed,product_id_CartStarted,last_cart_started
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,2017-05-19 08:02:51,None,None,False,None,True,2017-05-19 08:02:53+00:00,2017-05-19 08:02:53+00:00,1495182843,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN,3838997,2017-05-19 08:02:53
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,2017-05-19 08:02:51,None,None,False,None,True,2017-05-19 13:51:59+00:00,2017-05-19 13:51:59+00:00,1495203782,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN,3838997,2017-05-19 13:51:59
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=Title&blockpos=19&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-02&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9751833&st_rid=20405273,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,2017-06-02 21:40:06,None,9,False,None,False,2017-06-02 21:40:06+00:00,2017-06-02 21:40:06+00:00,1496441524,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN,4096138,2017-06-02 21:40:06
3,1,4,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=View&blockpos=1&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-09&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9805080&st_rid=20405273,NaT,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,2017-06-09 14:23:10,None,9,False,None,False,2017-06-09 14:23:10+00:00,2017-06-09 14:23:10+00:00,1497020106,77.103.157.226,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1",None,GB,NaN,4096138,2017-06-09 14:23:10
4,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https://www.wowcher.co.uk/email-deals/travel/4009686/69pp-weekender-breaks-3nt-4-algarve?Linkpos=Image&blockpos=9&deal=4009686&ito=wowcher_email_Travel_26052017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9703436&st_rid=5555079,NaT,2017-05-26 13:25:20.891,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,2017-05-26 13:25:20,None,69,False,None,False,2017-05-26 13:25:27+00:00,2017-05-26 13:25:27+00:00,1495807024,148.252.216.66,"Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/49.0.2623.112+Safari/537.36",None,GB,NaN,4009686,2017-05-26 13:25:27
5,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https://www.wowcher.co.uk/email-deals/travel/4009686/69p

---
# Appendix

In [ ]:
# itertuples example for adding num_sess. 
# Not efficient at all, only 2 or 3 iterations/second
for tuple in tqdm(data0.itertuples()):
    i = tuple[0]+1
    # if id is same:
    if data0.loc[i,'id'] == data0.loc[i-1,'id']:
        # session_id is same:
        if data0.loc[i,'session_id']==data0.loc[i-1,'session_id']:  
            data0.loc[i,'num_sess']=data0.loc[i-1,'num_sess']
        # session_id is different:
        else:
            data0.loc[i,'num_sess']=data0.loc[i-1,'num_sess']+1
    else:
    # session_id is different:
      data0.loc[i,'num_sess']=1

In [16]:
data[['id','session_id']].apply(lambda x: str(x))

<bound method NDFrame.head of id            0              1\n1              1\n2         ...
session_id    0            k79c9lk1nfop9m4wx0fmrg16\n1      ...
dtype: object>

In [ ]:
# for loop example for adding num_sess.
data=data[:5000]
data['num_sess']=1

for i in trange(1,data.shape[0]):
    # if id is same:
    if data.iloc[i,0] == data.iloc[i-1,0]:   
        # session_id is same:
        if data.iloc[i,1]==data.iloc[i-1,1]:  
            data.iloc[i,-1]=data.iloc[i-1,-1]
        # session_id is different:
        else:
            data.iloc[i,-1]=data.iloc[i-1,-1]+1
    else:
        # session_id is different:
        data.iloc[i,-1]=1

In [61]:
# test=df.groupby(['id','num_sess'])['total'].nunique()

In [62]:
# test[test.values!=1]

id      num_sess
1       9           2
835     3           2
841     3           0
1257    3           2
1347    4           2
1663    4           2
1707    3           2
3128    1           0
3340    2           0
3675    3           0
3942    6           0
4398    1           0
4696    4           0
4706    2           2
4951    2           0
5008    6           2
5071    3           0
5428    1           0
5858    5           2
5979    2           2
5983    3           0
6089    1           0
6241    1           0
6807    3           2
8644    1           0
8741    2           0
10007   2           0
10074   1           2
10245   2           0
10822   2           2
                   ..
170791  1           2
181911  1           0
183803  1           2
185679  1           0
187850  1           2
189592  1           0
190489  1           2
196221  1           0
196672  1           0
198317  1           2
198982  1           0
201278  1           2
202160  1           2
202470  1      

In [68]:
sum(test.values!=1)

178

In [128]:
df0.groupby(['index','num_sess'])